## Load the Data

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))  # Flatten 28x28 to 784
])

train_dataset = datasets.MNIST(root='./data',train=True,transform=transform,download=True)
test_dataset = datasets.MNIST(root='./data',train=False,transform=transform)

train_loader = DataLoader(train_dataset,batch_size=64,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=64,shuffle=False)


100%|██████████| 9.91M/9.91M [00:03<00:00, 3.21MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 157kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.52MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.67MB/s]


## Define Neural Network

In [3]:
class FeedforwardNN(nn.Module):
  def __init__(self):
    super(FeedforwardNN,self).__init__()
    self.fc1 = nn.Linear(784,256)
    self.fc2 = nn.Linear(256,128)
    self.fc3 = nn.Linear(128,10)
  def forward(self,x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = self.fc3(x)
    return x

## Training Loop

In [4]:
model = FeedforwardNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

In [6]:
epochs = 5
for epoch in range(epochs):
  model.train()
  total_loss = 0
  correct = 0
  total = 0

  for images, labels in train_loader:

    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
  accuracy = correct / total
  print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}, Train Accuracy: {accuracy:.4f}")


Epoch 1, Loss: 28.7318, Train Accuracy: 0.9901
Epoch 2, Loss: 22.4823, Train Accuracy: 0.9924
Epoch 3, Loss: 20.3290, Train Accuracy: 0.9928
Epoch 4, Loss: 16.2156, Train Accuracy: 0.9942
Epoch 5, Loss: 14.5458, Train Accuracy: 0.9950


## Evaluate

In [7]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
  for images, labels in test_loader:
    outputs = model(images)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.9803
